# Метод двойственных усреднений поиска равновесия в модели стабильной динамики (Нестерова-де Пальма)

## Импорт библиотек

In [6]:
import numpy as np
import codecs
import re

## Чтение данных

In [7]:
data_path = '../data/SiouxFalls_trips.txt'

massiv = np.zeros((24,24))
with codecs.open(data_path, 'r', 'utf-8') as fin:
    for line_idx, line in enumerate(fin):
        row = line.strip('\n').split(' ')
        if len(row) == 3:
            i = int(row[1])-1
            j = 0
        else:
            for x in row:
                t = re.findall(r'(\w+.0);', x)
                if len(t) != 0:
                    massiv[i][j] = float(t[0]) 
                    j += 1
massiv = massiv.tolist()


data_path2 = '../data/SiouxFalls_net.txt'

init = []
term = []
A = []
C = []
t0 = []
f0 = []

with codecs.open(data_path2, 'r', 'utf-8') as fin:
    for line_idx, line in enumerate(fin):
        if line_idx == 0:
            continue
        row = line.strip('\n').split('\t')

        init.append(int(row[1])-1)
        term.append(int(row[2])-1)
        A.append(int(row[5]))
        if int(row[7]) == 0:
            C.append(0)
        else:
            C.append(int(row[5]) * float(row[6]) / (float(row[3]) ** int(row[7])))
        t0.append(int(row[5]))
        f0.append(float(row[3]))

##  Начальные данные

In [8]:
nodes = 24
Links = 76

t = [0]*Links
f = [0]*Links

t = t0

epsilon = 0
for e in range(Links):
    epsilon = epsilon + A[e] * f0[e] + C[e] * (f0[e] ** 5) / 5.
epsilon = 0.01 * epsilon

M = [0]*1000
gamma = [0]*1000
SN = 0
Arg1 = [0]*Links
Arg2 = [0]*Links
tN = [0]*Links

GAMMA = 10**10
PSI = 10**10

## Метод двойственных усреднений для предельного случая

В качестве критерия останова используем: 
$$\Gamma(\overline{t}^N) + \Psi(\overline{f}^N) \leq \epsilon,$$
где $$\Psi(f) = \sum\limits_{e \in E} f_e \overline{t}_e,$$
$$\Gamma(t) = -\sum\limits_{w \in W} d_w T_w(t) + (\overline{f}, t-\overline{t}),$$
$T_w(t)$ - длина кратчайшего пути из $i$ в $j$ ($w=(i,j) \in W$ - корреспонденции) на графе, ребра которого взвешаны вектором $t=\{t_e\}_{e \in E}$. 

Для решения используем метод зеркального спуска:
$$t^{k+1} = \arg \min_{t_e \geq \overline{t}_e, e \in E} \{\gamma_k (\partial F(t^k), t-t^k) + \frac{1}{2} || t-t^k||^2_2\},$$
где $F(t) = -\sum\limits_{w \in W} d_w T_w(t),$
$$\gamma_k = \frac{\epsilon}{M_k^2},\,\, M_k = ||\partial F(t^k)||_2.$$

$$\overline{t}^N = \frac{1}{S_N}\sum\limits_{k=0}^N \gamma_k t^k,\,\, S_N=\sum\limits_{k=0}^N \gamma_k,$$

$$\overline{f}^N_e = \overline{f}_e-s_e^N, \,\, e \in E,$$

где $s_e \geq 0$ - множитель Лагранжа к ограничению $t_e \geq \overline{t}_e$ в задаче

$$\frac{1}{S_N} \{ \sum\limits_{k=0}^N \gamma_k \sum\limits_{e \in E} \{ (\partial_e F(t^k), t_e-t_e^k)\} + S_N \sum\limits_{e \in E} \overline{f}_e (t_e-\overline{t}_e + \frac{1}{2}\sum\limits_{e \in E} (t_e-\overline{t}_e)^2\} \rightarrow \min\limits_{t_e \geq \overline{t}_e, e \in E}.$$

In [11]:
#Номер итерации 
k = 0
#Проверяем критерий останова 
while (GAMMA + PSI >= epsilon or GAMMA + PSI < 0):

    #Массив для хранения текущего решения
    y=[0]*Links;

    #Дополнительный массив для вычисления субградиента PSI
    delta = [0]*nodes
    for i in range(nodes):
        delta[i] = [0]*nodes
        for j in range(nodes):
            delta[i][j] = [0]*Links 


    #Создаем массив весов ребер для алгоритма Дейкстры
    w = [0]*nodes
    for i in range(nodes):
        w[i] = [10 ** 10]*nodes

    for e in range(Links):
        w[init[e]][term[e]] = t[e];

    #Алгоритм Дейкстры (сложность O(n^2))
    for i in range(nodes):

        INF = 10 ** 10
        p = [-1] * nodes
        dist = [INF] * nodes
        dist[i] = 0
        used = [False] * nodes
        min_dist = 0
        min_vertex = i
        while min_dist < INF:
            s = min_vertex 
            used[s] = True 
            for j in range(nodes): 
                if dist[s] + w[s][j] < dist[j]: 
                    dist[j] = dist[s] + w[s][j] 
                    p[j] = s;
            min_dist = INF
            for j in range(nodes):
                if not used[j] and dist[j] < min_dist:
                    min_dist = dist[j]
                    min_vertex = j


        for j in range(nodes):
            s = p[j]
            ss = j
            while (s!=-1):
                for e in range(Links):
                    if ((init[e] == s) and (term[e] == ss)):
                        delta[i][j][e]=1
                ss = p[ss]
                s = p[s]
    
    a = 1.0/2

    #Дополнительный параметр для вычисления субградиента PSI
    Sum = [0]*Links

    for e in range(Links):
        for i in range(nodes):
            for j in range(nodes):
                Sum[e] = Sum[e] + massiv[i][j] * delta[i][j][e]

    M[k] = 0
    for e in range(Links):
        M[k] = M[k] + (f0[e] - Sum[e]) ** 2

    gamma[k] = float(epsilon) / M[k]

    #Вычисляем новое решение задачи минимизации - y[k]
    for e in range(Links):

        b = - t[e] + gamma[e] * (f0[e] - Sum[e])
        c = 1.0/2 * (t[e] ** 2) - gamma[e] * t[e] * Sum[e]


        if (-b <= t0[e]):
            y[e] = t0[e]
            lambd = t0[e] + b
        elif (a*t0[e]*t0[e] + b*t0[e] > -1.0/2 * b * b):
            y[e] = -b
            lambd = 0
        else:
            y = t0[e]
            lambd = t0[e] + b

    if k >= 1:

        #Вычисляем параметры для критерия останова
        SN = SN + gamma[k]
        for e in range(Links):
            Arg1[e] = Arg1[e] + gamma[k] * (f0[e] - Sum[e])

            aa = 0.5 / SN
            bb = - (1. / SN) * t0[e] + f0[e] + (1. / SN) * Arg1[e]
            cc = - (1. / SN) * Arg1[e] * t[e] - f0[e] * t0[e] + (0.5 / SN) * (t0[e] ** 2)

            #lambd - множитель Лагранжа 
            if (-bb <= t0[e]):
                lambd = t0[e] + bb
            elif (aa*t0[e]*t0[e] + bb*t0[e] > -0.5 * bb * bb):
                lambd = 0
            else:
                lambd = t0[e] + bb


            f[e] = f0[e] - lambd


        for e in range(Links):
            Arg2[e] = Arg2[e] + gamma[k] * t[e]
            tN[e] = (1. / SN) * Arg2[e]

        #Еще раз применяем алгоритм Дейкстры для нахождения значения GAMMA
        GAMMA = 3 * 10**5
        for e in range(Links):
            w[init[e]][term[e]] = tN[e];

        for i in range(nodes):
            INF = 10 ** 10
            dist = [INF] * nodes
            dist[i] = 0
            used = [False] * nodes
            min_dist = 0
            min_vertex = i
            while min_dist < INF:
                s = min_vertex 
                used[s] = True 
                for j in range(nodes): 
                    if dist[s] + w[s][j] < dist[j]: 
                        dist[j] = dist[s] + w[s][j] 
                        p[j] = s;
                min_dist = INF
                for j in range(nodes):
                    if not used[j] and dist[j] < min_dist:
                        min_dist = dist[j]
                        min_vertex = j

            for j in range(nodes):
                GAMMA = GAMMA - massiv[i][j] * dist [j]

        #Вычисляем значение PSI
        PSI = 0
        for e in range(Links):
            PSI = PSI + A[e] * f[e] + C[e] * (f[e] ** 5) / 5.


    #Решение на текцщем шаге
    t = y

    #Новый шаг
    k = k + 1



## Вывод результатов

In [12]:
print ('Дуга: оптимальная величина потока\n')
for e in range(Links):
    print ('%d %d: f = %d' % (init[e], term[e], f[e]))

Дуга: оптимальная величина потока

0 1: f = 25900
0 2: f = 18261
1 0: f = 25900
1 5: f = 4958
2 0: f = 18261
2 3: f = 17110
2 11: f = 20281
3 2: f = 17110
3 4: f = 13685
3 10: f = 4908
4 3: f = 13982
4 5: f = 4947
4 8: f = 10000
5 1: f = 4958
5 4: f = 4947
5 7: f = 4898
6 7: f = 7841
6 17: f = 7553
7 5: f = 4898
7 6: f = 7841
7 8: f = 5050
7 15: f = 5045
8 4: f = 10000
8 7: f = 5050
8 9: f = 13915
9 8: f = 13915
9 10: f = 10000
9 14: f = 13512
9 15: f = 4854
9 16: f = 4993
10 3: f = 4908
10 9: f = 10000
10 11: f = 4908
10 13: f = 4876
11 2: f = 19061
11 10: f = 4908
11 12: f = 12848
12 11: f = 12948
12 23: f = 5091
13 10: f = 4876
13 14: f = 5127
13 22: f = 4924
14 9: f = 13512
14 13: f = 5127
14 18: f = 14564
14 21: f = 9599
15 7: f = 5045
15 9: f = 4854
15 16: f = 5229
15 17: f = 19679
16 9: f = 4993
16 15: f = 5229
16 18: f = 4823
17 6: f = 7571
17 15: f = 19679
17 19: f = 23403
18 14: f = 14564
18 16: f = 4823
18 19: f = 5002
19 17: f = 23403
19 18: f = 5002
19 20: f = 5059
19 21: 